In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, 
                          Activation, Dropout, Conv2D, Conv2DTranspose,
                          Concatenate, Add, Multiply)
from flows import MaskingDense, IAF, MAF
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
img_rows, img_cols, img_chns = 28, 28, 1
original_img_size = (img_rows, img_cols, img_chns)
batch_size = 256
# 生成的z的维度
latent_dim = 16

epochs = 1000
learning_rate = 0.0005
dropout = 0.1

# 8000太大了，内存不够，设置成一半
hidden_units = 4000
# masking dense的层数
hidden_layers = 3


原始的数据读取。模仿这个方法进行读取。

然后再看看数据集的数据是什么样子的。

构造输入，将其变成batch_size * len的形式，输出是评分。

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train.astype(np.float32), X_test.astype(np.float32)

X_train = np.round(X_train.reshape(X_train.shape[0], img_rows * img_cols) / 255.)
X_test = np.round(X_test.reshape(X_test.shape[0], img_rows * img_cols) / 255.)

# (60000, 784) 的x_train切片，切成60000份，每个train_data都是 784的大小
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# repeat() 相当于一个dataset训练两遍，batch是一次取出256个image的数据，prefetch(1)是每次预存一个batch，shuffle指一个pool有多大
train_data = train_data.repeat().batch(batch_size).prefetch(1)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.repeat().batch(batch_size).prefetch(1)

11493376/11490434 [==============================] - 2s 0us/step


user_artists 的结构：

    userID artistID weight

这个weight应该是访问的次数，或者叫收听的次数。这个范围非常大，而CAf做的是将user和item编码，然后得到rating，所以这个结果应该是0-1之间的，符合二元交叉熵的。要有一个方案，将这个weight归一化到这个范围内。

每个userID有50个数据

参考CAF的结论，Last FM的user有 1892 个，item有17632个，interaction有92,834，Sparsity是99.7% 100 - 92834/ (1892 * 17632) *100

movielen的是95.4% 100 - 993482 / (6040 * 3544) * 100

user用item来做特征，反之亦然。

找到最受欢迎的20%歌手，组成每个user的原始交互向量，维度是3500，可以直接省略成3000。因为one-hot实在是太大了，如果要one-hot，维度是17632。

item可以one-hot，维度是1892。但是item有更好的tag系统，也使用最火的20%tag来做标记，维度是2500。

或者都用tag？

In [15]:
dataset_path = r"C:\Users\i9233\recommend system\dataset\hetrec2011-lastfm-2k"
hottest = int(math.pow(25, 2))

In [16]:
# user 对 artist的tag
user_tag = pd.read_table(dataset_path + "\\user_taggedartists.dat")
# 空的dataframe
hot_tag = pd.DataFrame(columns=['tagID', 'sum'])

In [17]:
g = user_tag.groupby('tagID').groups
i = 0
for k in g.keys():
    hot_tag.loc[i] = {'tagID':k, 'sum':len(g[k])}
    i = i+1
# 统计不同的tag的使用频率

In [18]:
# 找到最火的hottest_tag个tag
s = hot_tag.sort_values("sum",ascending=False, inplace=False).reset_index(drop=True)

In [19]:
# 重新按照tagid进行排序，并输出到csv中
h = s[:hottest].sort_values("tagID", ascending=True, inplace=False).reset_index(drop=True)
h.to_csv(dataset_path + "\\hottest_tag_" + str(hottest) + '.csv', index=False)

In [20]:
hottest_tag = pd.read_csv(dataset_path + "\\hottest_tag_" + str(hottest) + ".csv", )
hottest_tag

,tagID,sum
0,1,1729
1,2,212
2,4,301
3,5,582
4,6,210
...,...,...
620,12218,42
621,12219,37
622,12265,38
623,12287,71


In [21]:
1 in hottest_tag['tagID']

True

In [22]:
hottest_tag[hottest_tag['tagID']==1].index.to_numpy()

array([0], dtype=int64)

读数据的时候，user看过哪些电影，就按照这个电影的tag对user的向量进行赋值，即user长hottest_tag。

假如user看过singer a，那么找到a的全部tag，然后按照这个tag对weight进行累积，全部singer统计完后，对tag的weight进行sigmoid，得到user对某个tag的感兴趣程度。

In [23]:
user_artists = pd.read_table(dataset_path + "\\user_artists.dat")
user_artists

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


In [24]:
user_tag

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009
...,...,...,...,...,...,...
186474,2100,16437,4,1,7,2010
186475,2100,16437,292,1,5,2010
186476,2100,16437,2087,1,7,2010
186477,2100,16437,2801,1,5,2010


In [25]:
user_artists_ids = user_artists.groupby('userID').groups

In [26]:
# id = artist_id的artist对应的全部tag
artist_tag = user_tag.groupby('artistID').groups

下面开始得到全部的可训练的user向量。

迭代每一个user，对user订阅的每个artist，都寻找这个artist的全部tag，检查tag是否在hottest里面，如果在，则在user对应的列，加上artist的权重，循环完全部的artist可以得到user的全部权重。

item的和user类似，少一个循环。

迭代每一个item，也就是artist，找到全部被打上的tag，检查tag是否在hottest里面，如果在，则在item对应的地方，加上1，可能有很多人都给这个item打了同一个tag，循环完全部的item即可得到item的向量。

In [30]:
# 这块代码需要40min
all_user = []
for user_id in user_artists_ids.keys():
    # 得到user_id 对应的全部 artist id
    artist_list = user_artists.loc[user_artists_ids[user_id]]
    artist_list = artist_list['artistID'].to_numpy()

    for artist_id in artist_list:
        # 得到artist_id 对应的全部tag，并去重
        if artist_id not in artist_tag.keys():
            continue
        indexes = artist_tag[artist_id]
        tag_list = user_tag.loc[indexes]['tagID'].drop_duplicates(inplace=False).to_numpy()
        
        # 准备一个空的vec
        user_vec = np.zeros(shape=[hottest])
        
        for tag_id in tag_list:
            # tag_id是否在hottest_tag里，如果在，那么user_vec的相对位置上加上这个artist_id的weight
            if tag_id in hottest_tag['tagID'].values:
                # tag_id -> vec_id
                vec_id = hottest_tag[hottest_tag['tagID']==tag_id].index[0]
                temp = user_artists[user_artists['artistID'] == artist_id]
                
                user_vec[vec_id] += temp[temp['userID'] == user_id]['weight'].values[0]
        
    all_user.append(user_vec)

In [31]:
pd_vec = pd.DataFrame(all_user)
pd_vec

,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,193.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,115.0,115.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0,0.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
test = pd_vec.loc[0:5]

In [57]:
test

,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,193.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,115.0,115.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,498.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
1/(np.exp(-test)+1)

,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1.0,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5
3,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1.0,1.0,1.0,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1.0
4,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [32]:
pd_vec.to_csv(dataset_path + "\\user_representation.csv", index=False)

迭代每一个item，也就是artist，找到全部被打上的tag，检查tag是否在hottest里面，如果在，则在item对应的地方，加上1，可能有很多人都给这个item打了同一个tag，循环完全部的item即可得到item的向量。

In [27]:
artists_ids = user_tag.groupby('artistID').groups
all_item = []
for artists_id in artists_ids.keys():
    # 迭代每一个artist，找到被打上的tag
    tag_list = user_tag.loc[artists_ids[artists_id]]
    tag_list = tag_list['tagID'].values
    
    item_vec = np.zeros(shape=[hottest])
    for tag_id in tag_list:
        if tag_id in hottest_tag['tagID'].values:
            vec_id = hottest_tag[hottest_tag['tagID']==tag_id].index.values[0]
            item_vec[vec_id] += 1 
    
    all_item.append(item_vec)

In [28]:
pd_vec = pd.DataFrame(all_item)
pd_vec

,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
pd_vec.to_csv(dataset_path + "\\item_representation.csv", index=False)